In [1]:
!pip install \
  tiktoken==0.4.0 \
  openai==0.27.7 \
  langchain==0.0.179 \
  pinecone-client \
  datasets==2.13.1

In [2]:
import pandas as pd

/var/folders/dc/0gdv70ds3ll_w8qb4v23h0c00000gn/T/ipykernel_55051/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
df = pd.read_csv('../scraper/data.csv')
df_structured = pd.read_csv('../scraper/structured_data.csv')

In [3]:
df_structured.head()

,PS_number,manufacturer_number,description,reviews,troubleshooting,repair_stories,q_a
0,PS11701542,EDR1RXD1,Product Description Refrigerator Ice and...,"Fast delivery:\n\nPart came in fast, I install...",Troubleshooting This part fixes the follo...,Water ?? filter needed to be replaced:\n\nSimp...,"QUESTION #0: Your water filter w10295370a , wh..."
1,PS12364199,242126602,Product Description Refrigerator Door Sh...,Great quality parts.:\n\nThe part was brand ne...,Troubleshooting This part works with the ...,The door bin on my Refrigerator broke:\n\nI re...,QUESTION #0: Is this a replacement for the bot...
2,PS16542496,WE03X29897,Product Description BELT DRIVE Specifica...,Easy repair:\n\nThe part fit perfectly. Had my...,Troubleshooting This part fixes the follo...,"Dryer was making increasingly loud ""thumping"" ...",QUESTION #0: is motor supposed to stay running...
3,PS11752778,WPW10321304,Product Description Refrigerator Door Sh...,Easy repair. Part fit perfectly.:\n\nPart wor...,Troubleshooting This part fixes the follo...,Refrigerator door shelf and door shelf bin are...,QUESTION #0: How do I replace this bin\nANSWER...
4,PS438018,316075103,Product Description Oven Bake Element Sp...,Works:\n\nTook a few minutes and had it swappe...,Troubleshooting This part fixes the follo...,Electrical connection pulled off the element:\...,QUESTION #0: I think it would be good to tell ...


In [4]:
df.head()

,item_name,PS_number,manufacturer,manufacturer_number,text,url
0,Refrigerator Ice and Water Filter,PS11701542,Whirlpool,EDR1RXD1,...,https://www.partselect.com/PS11701542-Whirlpoo...
1,Refrigerator Door Shelf Bin,PS12364199,Frigidaire,242126602,...,https://www.partselect.com/PS12364199-Frigidai...
2,Refrigerator BELT DRIVE,PS16542496,GE,WE03X29897,...,https://www.partselect.com/PS16542496-GE-WE03X...
3,Refrigerator Door Shelf Bin,PS11752778,Whirlpool,WPW10321304,...,https://www.partselect.com/PS11752778-Whirlpoo...
4,Refrigerator Oven Bake Element,PS438018,Frigidaire,316075103,...,https://www.partselect.com/PS438018-Frigidaire...


In [4]:
import tiktoken

tokenizer_name = tiktoken.encoding_for_model('gpt-4')
tokenizer = tiktoken.get_encoding(tokenizer_name.name)

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)

In [5]:
from typing_extensions import Concatenate
from uuid import uuid4
from tqdm.auto import tqdm

chunks = []

for j, page in tqdm(df.iterrows()):
    if len(page['text']) < 200:
        # if page content is short we can skip
        continue
    texts = text_splitter.split_text(page['text'])
    chunks.extend([{
        'id': page['item_name'] + f'-{i}',
        'text': texts[i],
        'url': page['url'],
        'chunk': i
    } for i in range(len(texts))])
len(chunks)

0it [00:00, ?it/s]

551

In [7]:
!openai migrate

Retrieving Grit CLI metadata from https://api.keygen.sh/v1/accounts/custodian-dev/artifacts/marzano-macos-arm64
Fetching release URL from: https://api.keygen.sh//v1/accounts/custodian-dev/artifacts/cli-macos-arm64
Fetching release URL from: https://api.keygen.sh//v1/accounts/custodian-dev/artifacts/workflow_runner-macos-arm64
Fetching release URL from: https://api.keygen.sh//v1/accounts/custodian-dev/artifacts/marzano-macos-arm64
Fetching release URL from: https://api.keygen.sh//v1/accounts/custodian-dev/artifacts/gouda-macos-arm64
Fetching release URL from: https://api.keygen.sh//v1/accounts/custodian-dev/artifacts/timekeeper-macos-arm64
Your working tree currently has untracked changes and Grit will rewrite files in place. Do you want to proceed? [y/N] 

In [6]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()

# get API key from top-right dropdown on OpenAI website
openai.api_key = os.getenv("OPENAI_API_KEY") or "OPENAI_API_KEY"

openai.Engine.list()  # check we have authenticated

APIRemovedInV1: 

You tried to access openai.Engine, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [6]:
embed_model = "text-embedding-ada-002"

res = openai.Embedding.create(
    input=[
        "Sample document text goes here",
        "there will be several phrases in each batch"
    ], engine=embed_model
)

In [9]:
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.getenv("PINECONE_API_KEY") or "PINECONE_API_KEY"
# find your environment next to the api key in pinecone console
# env = os.getenv("PINECONE_ENVIRONMENT") or "PINECONE_ENVIRONMENT"
pc = Pinecone(api_key=api_key)
pinecone = pc

In [16]:
index_name = 'gpt-4-part-data2'

In [36]:
import time
from pinecone import ServerlessSpec

# check if index already exists (it shouldn't if this is first time)
if index_name not in pinecone.list_indexes().names():
    # if does not exist, create index
    pinecone.create_index(
        index_name,
        dimension=len(res['data'][0]['embedding']),
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-west-2'
        )
    )
    # wait for index to be initialized
    while not pinecone.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pinecone.Index(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [37]:
from tqdm.auto import tqdm

batch_size = 100  # how many embeddings we create and insert at once

for i in tqdm(range(0, len(chunks), batch_size)):
    # find end of batch
    i_end = min(len(chunks), i+batch_size)
    meta_batch = chunks[i:i_end]
    # get ids
    ids_batch = [x['id'] for x in meta_batch]
    # get texts to encode
    texts = [x['text'] for x in meta_batch]
    # create embeddings (try-except added to avoid RateLimitError)
    try:
        res = openai.Embedding.create(input=texts, engine=embed_model)
    except:
        done = False
        while not done:
            time.sleep(5)
            try:
                res = openai.Embedding.create(input=texts, engine=embed_model)
                done = True
            except:
                pass
    embeds = [record['embedding'] for record in res['data']]
    # cleanup metadata
    meta_batch = [{
        'text': x['text'],
        'chunk': x['chunk'],
        'url': x['url']
    } for x in meta_batch]
    to_upsert = list(zip(ids_batch, embeds, meta_batch))
    # upsert to Pinecone
    index.upsert(vectors=to_upsert)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:06<00:00, 11.16s/it]


In [45]:
import re
PS_NUMBER_PATT = r'\b(PS\d{8})\b'
MANUFACTURER_NUMBER_PATT = r'\b[A-Z0-9]{4,11}\b'
def augment_query(query, df):
    ps_search = set(re.findall(PS_NUMBER_PATT, query))
    masked_df = df[df['PS_number'].isin(ps_search)]
    ps_augment = "\n\n---\n\n".join(map(lambda t: t.strip(), masked_df['text']))
    augmented = set(masked_df['manufacturer_number'].values)
    manufacturer_search = set(re.findall(MANUFACTURER_NUMBER_PATT, query)) - ps_search - augmented
    manufacturer_augment = "\n\n---\n\n".join(map(lambda t: t.strip(), df[df['manufacturer_number'].isin(manufacturer_search)]['text']))
    return ps_augment+manufacturer_augment+"\n\n-----\n\n"+query
def augment_query2(query, df):
    ps_search = set(re.findall(PS_NUMBER_PATT, query))
    masked_df = df[df['PS_number'].isin(ps_search)]
    ps_augment = f"CONTEXT FOR {','.join(ps_search)}:\n"+ "\n\n---\n\n".join(map(lambda t: t.strip(), masked_df['description']))
    # ps_augment += "\n\n---\n\n".join(map(lambda t: t.strip(), masked_df['reviews']))
    # ps_augment += "\n\n---\n\n".join(map(lambda t: t.strip(), masked_df['troubleshooting']))
    # ps_augment += "\n\n---\n\n".join(map(lambda t: t.strip(), masked_df['repair_stories']))
    ps_augment += "\n\n---\n\n".join(map(lambda t: t.strip(), masked_df['q_a']))
    # augmented = set(masked_df['manufacturer_number'].values)
    # manufacturer_search = set(re.findall(MANUFACTURER_NUMBER_PATT, query)) - ps_search - augmented
    # manufacturer_augment = "\n\n---\n\n".join(map(lambda t: t.strip(), df[df['manufacturer_number'].isin(manufacturer_search)]['text']))
    return ps_augment+"\n\n-----\n\n"+query

In [44]:
query = "How can I install part number PS11752778?"
# query = "The ice maker on my Whirlpool fridge is not working. How can I fix it?"

res = openai.Embedding.create(
    input=[query],
    engine=embed_model
)

# retrieve from Pinecone
xq = res['data'][0]['embedding']

# get relevant contexts (including the questions)
res = index.query(vector=xq, top_k=5, include_metadata=True)

In [45]:
res

{'matches': [{'id': 'Refrigerator Water Filter-16',
              'metadata': {'chunk': 16.0,
                           'text': 'Thank you for voting!      Jones  July 20, '
                                   '2017   My icemaker just stopped making ice '
                                   'and I know I need to replace the water '
                                   'filter ive just been dragging my feet '
                                   'about it. Is the icemaker cut off now '
                                   'because I need to replace my filter or is '
                                   'there another reason for it stopping '
                                   'now?     Hello Jones, we recommend '
                                   'replacing the water filter every 6 months, '
                                   'depending on how often you use it. '
                                   'Replacing or not replacing the filter '
                                   'would only affect the

In [46]:
# get list of retrieved text
contexts = [item['metadata']['text'] for item in res['matches']]

augmented_query = "\n\n---\n\n".join(contexts)+"\n\n-----\n\n"+query

In [47]:
print(augmented_query)

Thank you for voting!      Jones  July 20, 2017   My icemaker just stopped making ice and I know I need to replace the water filter ive just been dragging my feet about it. Is the icemaker cut off now because I need to replace my filter or is there another reason for it stopping now?     Hello Jones, we recommend replacing the water filter every 6 months, depending on how often you use it. Replacing or not replacing the filter would only affect the water dispenser, not the icemaker. The most common reason an icemaker does not get water would be a defective water valve WPW10270395. I hope this helps.    34 people found this helpful.   Was this helpful?      Thank you for voting!      Ivan  February 18, 2018   I just replaced my water filter on my Whirlpool gold however its making ice but its not dispensing water or ice. What van i do?  For model number G15SVAXVL01     Hi Ivan,

---

Back to Top      Customer Repair Stories    Average Repair Rating: 4.9 / 5.0, 114 reviews. What's this?  

In [48]:
# system message to 'prime' the model
primer = f"""You are Q&A bot. A highly intelligent system that answers
user questions based on the information provided by the user above
each question. If the information can not be found in the information
provided by the user you truthfully say "I don't know".
"""

res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": augmented_query}
    ]
)

In [20]:
from IPython.display import Markdown

display(Markdown(res['choices'][0]['message']['content']))

I don't know.

In [50]:
# system message to 'prime' the model
primer = f"""You are Q&A bot. A highly intelligent system that answers
user questions based on the information provided by the user above
each question. If the information can not be found in the information
provided by the user you truthfully say "I don't know".
"""

res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": query}
    ]
)

In [51]:
display(Markdown(res['choices'][0]['message']['content']))

There could be several reasons why the ice maker on your Whirlpool fridge is not working. Here are a few troubleshooting steps you can try:

1. Ensure that the ice maker is turned on. Check if the power switch or lever is set to the "On" position.
2. Make sure that the water supply is connected and turned on. Verify if the water line is not kinked or blocked.
3. Clean the ice maker and remove any ice or debris that might be blocking the ice maker's operation. You can also try manually defrosting it.
4. Check the water inlet valve for any clogs or obstructions. If necessary, clean or replace the valve.
5. Verify if the ice maker's temperature is set correctly. It should be around 0 degrees Fahrenheit (-18 degrees Celsius).
6. Check if the ice maker's bin or storage is full. If so, empty it to allow for fresh ice production.
7. Restart your fridge by unplugging it from the power source for a few minutes, then plugging it back in.

If these steps don't resolve the issue, it is recommended to consult the Whirlpool customer support or contact a professional appliance repair technician for further assistance.

In [47]:
query = "How can I install part number PS11752778?"
# system message to 'prime' the model
# primer = f"""You are Q&A bot. A highly intelligent system that answers
# user questions based on the information provided by the user above
# each question. If the information can not be found in the information
# provided by the user you truthfully say "I don't know".
# """
primer = f"""You are Q&A bot for an e-commerce site. You are a highly intelligent system that answers
customer questions about various products. You will first be provided with information about the product
the customer is inquiring about including the product description, reviews, general troubleshooting advice, 
repair stories from customers, and popular questions from other customers and their corresponding answers
scraped from the site. Use this information to answer the customer's question about this product.
If the information can not be found in the provided context, you should truthfully say "I don't know".
"""

primer = f"""You are Q&A bot for an e-commerce site. You are a highly intelligent system that answers
customer questions about various products. You will first be provided with information about the product
the customer is inquiring about including the product description, reviews, general troubleshooting advice, 
repair stories from customers, and popular questions from other customers and their corresponding answers
scraped from the site. Use this information to answer the customer's question about this product.
"""
augmented = augment_query2(query, df_structured)[:4000] + '\n\n-------\n\n' + query
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": augmented}
    ]
)

In [46]:
augment_query2(query, df_structured)[:4000] + '\n\n-------\n\n' + query

'CONTEXT FOR PS11752778:\nProduct Description      Refrigerator Door Shelf Bin Specifications  This manufacturer-certified Refrigerator Door Shelf Bin, also known as the crisper bin and door shelf bin, is made of clear plastic, and is easy to install. It is used to hold items in the fridge door, like jars, bottles, cans, and condiments, and this item is clear with white trim around its top edge and you\'ll be able to visually see if the part is broken. This particular shelf attaches to the fridge door. You can check if this model is right for your appliance by checking your user manual and model number. This individually-sold door bin fits many different models of side-by-side refrigerator, and can be installed with the use of no tools.          How Buying OEM Parts Can Save You Time and MoneyQUESTION #0: How do I replace this bin\nANSWER #0: Hello Casey, thanks for reaching out to us. If the old bin is still in place, simply grab onto it with both hands and lift it away from the door,

In [48]:
from IPython.display import Markdown
display(Markdown(res['choices'][0]['message']['content']))

To install part number PS11752778, follow these steps:

1. If the old bin is still in place, grab onto it with both hands and lift it away from the door. It should come right off.

2. Line up the new bin where the old bin was and simply drop it into place.

That's it! Your new bin should now be successfully installed. Good luck with your repair.

### Multiple Pinecone Indices

In [10]:
from tqdm.notebook import tqdm
from tqdm.auto import tqdm
import time
from pinecone import ServerlessSpec
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500//2,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)

TEXT_TYPES = {'description', 'reviews', 'troubleshooting', 'repair_stories', 'q_a'}
# assert TEXT_TYPES == set(df_structured.columns)
def populate_vector_db(df_structured):
    DB_LOOKUP = {}
    for i, row in tqdm(df_structured.iterrows()):
        ps_number, manufacturer_number = row['PS_number'], row['manufacturer_number']
        index_name = f'{ps_number}-{manufacturer_number}-{round(time.time())}'.lower()
        pinecone.create_index(
        index_name,
        dimension=1536,
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-west-2'
            )
        )
        # wait for index to be initialized
        while not pinecone.describe_index(index_name).status['ready']:
            time.sleep(1)
        index = pinecone.Index(index_name)
        DB_LOOKUP[ps_number] = index
        DB_LOOKUP[manufacturer_number] = index

        chunks = []
        for text_type in TEXT_TYPES:
            offset = len(chunks)
            texts = text_splitter.split_text(row[text_type])
            chunks.extend([{
                'id': row['PS_number'] + f'-{i+offset}',
                'text': texts[i],
                'chunk': i + offset
            } for i in range(len(texts))])
        batch_size = 100  # how many embeddings we create and insert at once
        for i in range(0, len(chunks), batch_size):
            # find end of batch
            i_end = min(len(chunks), i+batch_size)
            meta_batch = chunks[i:i_end]
            # get ids
            ids_batch = [x['id'] for x in meta_batch]
            # get texts to encode
            texts = [x['text'] for x in meta_batch]
            # create embeddings (try-except added to avoid RateLimitError)
            try:
                res = openai.Embedding.create(input=texts, engine=embed_model)
            except:
                done = False
                while not done:
                    time.sleep(5)
                    try:
                        res = openai.Embedding.create(input=texts, engine=embed_model)
                        done = True
                    except:
                        pass
            embeds = [record['embedding'] for record in res['data']]
            # cleanup metadata
            meta_batch = [{
                'text': x['text'],
                'chunk': x['chunk'],
            } for x in meta_batch]
            to_upsert = list(zip(ids_batch, embeds, meta_batch))
            # upsert to Pinecone
            index.upsert(vectors=to_upsert)
    return DB_LOOKUP
DB_LOOKUP = populate_vector_db(df_structured)

0it [00:00, ?it/s]

In [31]:
import re
PS_NUMBER_PATT = r'\b(PS\d{8})\b'
MANUFACTURER_NUMBER_PATT = r'\b[A-Z0-9]{4,11}\b'
def augment_query(query, df):
    ps_search = set(re.findall(PS_NUMBER_PATT, query))
    masked_df = df[df['PS_number'].isin(ps_search)]
    ps_augment = "\n\n---\n\n".join(map(lambda t: t.strip(), masked_df['text']))
    augmented = set(masked_df['manufacturer_number'].values)
    manufacturer_search = set(re.findall(MANUFACTURER_NUMBER_PATT, query)) - ps_search - augmented
    manufacturer_augment = "\n\n---\n\n".join(map(lambda t: t.strip(), df[df['manufacturer_number'].isin(manufacturer_search)]['text']))
    return ps_augment+manufacturer_augment+"\n\n-----\n\n"+query
def augment_query2(query, df):
    ps_search = set(re.findall(PS_NUMBER_PATT, query))
    masked_df = df[df['PS_number'].isin(ps_search)]
    ps_augment = f"CONTEXT FOR {','.join(ps_search)}:\n"+ "\n\n---\n\n".join(map(lambda t: t.strip(), masked_df['description']))
    # ps_augment += "\n\n---\n\n".join(map(lambda t: t.strip(), masked_df['reviews']))
    # ps_augment += "\n\n---\n\n".join(map(lambda t: t.strip(), masked_df['troubleshooting']))
    # ps_augment += "\n\n---\n\n".join(map(lambda t: t.strip(), masked_df['repair_stories']))
    ps_augment += "\n\n---\n\n".join(map(lambda t: t.strip(), masked_df['q_a']))
    # augmented = set(masked_df['manufacturer_number'].values)
    # manufacturer_search = set(re.findall(MANUFACTURER_NUMBER_PATT, query)) - ps_search - augmented
    # manufacturer_augment = "\n\n---\n\n".join(map(lambda t: t.strip(), df[df['manufacturer_number'].isin(manufacturer_search)]['text']))
    return ps_augment+"\n\n-----\n\n"+query
def augment_query3(query, DB_LOOKUP, embed_model="text-embedding-ada-002"):
    res = openai.Embedding.create(
        input=[query],
        engine=embed_model
    )
    xq = res['data'][0]['embedding']
    ps_search = set(re.findall(PS_NUMBER_PATT, query)) & DB_LOOKUP.keys()
    ps_augment = ''
    if ps_search:
        # ps_augment = f"{','.join(ps_search)}:\n"
        for ps in ps_search:
            ps_augment += f'CONTEXT FOR {ps}:\n' + "\n\n---\n\n".join(map(lambda item: item['metadata']['text'].strip(), DB_LOOKUP[ps].query(vector=xq, top_k=5, include_metadata=True)['matches']))
        return ps_augment+"\n\n-----\n\n"+query
    else:
        manufacturer_search = set(re.findall(MANUFACTURER_NUMBER_PATT, query)) & DB_LOOKUP.keys()
        manufacturer_augment = ''
        for manufacturer_number in manufacturer_search:
            manufacturer_augment += f'CONTEXT FOR {manufacturer_number}:\n' + "\n\n---\n\n".join(map(lambda item: item['metadata']['text'].strip(), DB_LOOKUP[manufacturer_number].query(vector=xq, top_k=5, include_metadata=True)['matches']))
        return manufacturer_augment+"\n\n-----\n\n"+query
def augment_query4(query, DB_LOOKUP, embed_model="text-embedding-ada-002"):
    ps_search = set(re.findall(PS_NUMBER_PATT, query)) & DB_LOOKUP.keys()
    ps_augment = ''
    if ps_search:
        # ps_augment = f"{','.join(ps_search)}:\n"
        for ps in ps_search:
            ps_augment += f'CONTEXT FOR {ps}:\n' + "\n\n---\n\n".join(map(lambda item: item.page_content.strip(), DB_LOOKUP[ps].similarity_search(query, k=5)))
        return ps_augment+"\n\n-----\n\n"+query
    else:
        manufacturer_search = set(re.findall(MANUFACTURER_NUMBER_PATT, query)) & DB_LOOKUP.keys()
        manufacturer_augment = ''
        for manufacturer_number in manufacturer_search:
            manufacturer_augment += f'CONTEXT FOR {manufacturer_number}:\n' + "\n\n---\n\n".join(map(lambda item: item.page_content.strip(), DB_LOOKUP[manufacturer_number].similarity_search(query, k=5)))
        return manufacturer_augment+"\n\n-----\n\n"+query
    

In [26]:
DB_LOOKUP

{'PS11701542': <pinecone.data.index.Index at 0x1307ae7d0>,
 'EDR1RXD1': <pinecone.data.index.Index at 0x1307ae7d0>,
 'PS12364199': <pinecone.data.index.Index at 0x130bf83d0>,
 '242126602': <pinecone.data.index.Index at 0x130bf83d0>,
 'PS16542496': <pinecone.data.index.Index at 0x1310ddad0>,
 'WE03X29897': <pinecone.data.index.Index at 0x1310ddad0>,
 'PS11752778': <pinecone.data.index.Index at 0x130721990>,
 'WPW10321304': <pinecone.data.index.Index at 0x130721990>,
 'PS438018': <pinecone.data.index.Index at 0x104209e50>,
 '316075103': <pinecone.data.index.Index at 0x104209e50>,
 'PS734935': <pinecone.data.index.Index at 0x130e56dd0>,
 '240534901': <pinecone.data.index.Index at 0x130e56dd0>,
 'PS11722130': <pinecone.data.index.Index at 0x131000e50>,
 'EDR4RXD1': <pinecone.data.index.Index at 0x131000e50>,
 'PS11739119': <pinecone.data.index.Index at 0x13083ef50>,
 'WP2188656': <pinecone.data.index.Index at 0x13083ef50>,
 'PS755842': <pinecone.data.index.Index at 0x10420a6d0>,
 'WE1M504'

In [27]:
query = "Is this part compatible with my WPW10321304 model?"
augment_query3(query, DB_LOOKUP)+ '\n\n-------\n\n' + query

{'WPW10321304'}


'CONTEXT FOR WPW10321304:\nTroubleshooting     This part fixes the following symptoms: Door won’t open or close | Ice maker won’t dispense ice | Leaking   This part works with the following products: Refrigerator.   This part works with the following products: Whirlpool, Kenmore, Maytag, KitchenAid.   Part# WPW10321304 replaces these:  AP6019471,  2171046,  2171047,  2179574,  2179575,  2179607,  2179607K,  2198449,  2198449K,  2304235,  2304235K,  W10321302,  W10321303,  W10321304,  W10549739,  WPW10321304VP     Back to Top\n\n---\n\n---\n\nQUESTION #3: I have this model of Whirlpool refrigerator and the bin on the door is broken.  Will the model # wpw10321304 work for it?  Thanks.Sherie\nANSWER #3: Hello Sherie, thank you for inquiring. Door Bin PS11752778 is the refrigerator\'s Lower Door Bin. The Upper Door Bin is PS11752309.\n\n\n\n---\n\nQUESTION #4: Need the lower door bin. Please\nANSWER #4: Hello Dhonna, Thank you for contacting us. I have researched the model you have provide

In [31]:
query = "How can I install part number PS11752778?"
# query = "who won the 2010 nba finals?"
# query = "Is this part compatible with my WDT780SAEM1 model?"
# system message to 'prime' the model
primer = f"""You are Q&A bot for an e-commerce site. You are a highly intelligent system that answers
customer questions about various products. You will first be provided with information about the product
the customer is inquiring about including the product description, reviews, general troubleshooting advice, 
repair stories from customers, and popular questions from other customers and their corresponding answers
scraped from the site. Use this information to answer the customer's question about this product.
If the information can not be found in the provided context, you should truthfully say "I don't know".
"""

augmented = augment_query3(query, DB_LOOKUP2)
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": augmented}
    ]
)
from IPython.display import Markdown

display(Markdown(res['choices'][0]['message']['content']))

I don't know.

### FAISS

In [40]:
pip install -U langchain-community faiss-cpu langchain-openai tiktoken

  Obtaining dependency information for openai<2.0.0,>=1.10.0 from https://files.pythonhosted.org/packages/26/a1/75474477af2a1dae3a25f80b72bbaf20e8296191ece7fff2f67984206f33/openai-1.12.0-py3-none-any.whl.metadata
  Using cached openai-1.12.0-py3-none-any.whl.metadata (18 kB)
Using cached openai-1.12.0-py3-none-any.whl (226 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.28.1
    Uninstalling openai-0.28.1:
      Successfully uninstalled openai-0.28.1

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [41]:
!pip install langchain openai --upgrade

  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/3c/7c/138106d78cde8b994665251b0681ba59aede8e84902418bab8ea272f4585/langchain-0.1.7-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 6.6 MB/s eta 0:00:006.7 MB/s eta 0:00:01
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.316
    Uninstalling langchain-0.0.316:
      Successfully uninstalled langchain-0.0.316

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [20]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
import openai
from tqdm.notebook import tqdm
import time

TEXT_TYPES = {'description', 'reviews', 'troubleshooting', 'repair_stories', 'q_a'}
def populate_FAISS(df_structured):
    DB_LOOKUP = {}
    embeddings = OpenAIEmbeddings()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
        length_function=tiktoken_len,
        separators=["\n\n", "\n", " ", ""]
    )
    
    for i, row in tqdm(df_structured.iterrows()):
        ps_number, manufacturer_number = row['PS_number'], row['manufacturer_number']
        index_name = f'{ps_number}-{manufacturer_number}-{round(time.time())}'.lower()
        chunks = []
        for text_type in TEXT_TYPES:
            offset = len(chunks)
            texts = text_splitter.split_text(row[text_type])
            chunks.extend([{
                'id': row['PS_number'] + f'-{i+offset}',
                'text': texts[i],
                'chunk': i + offset
            } for i in range(len(texts))])
        print(chunks)
        db = FAISS.from_texts(texts, embeddings)  
        DB_LOOKUP[ps_number] = db
        DB_LOOKUP[manufacturer_number] = db
    return DB_LOOKUP

In [21]:
DB_LOOKUP2 = populate_FAISS(df_structured)

0it [00:00, ?it/s]

[{'id': 'PS11701542-0', 'text': 'Troubleshooting     This part fixes the following symptoms: Not dispensing water | Ice maker not making ice | Ice maker won’t dispense ice | Leaking   This part works with the following products: Refrigerator.   This part works with the following products: Whirlpool, Maytag, KitchenAid, Kenmore.   Part# EDR1RXD1 replaces these:  AP5982535,  9900,  9981,  FILTER 1,  FILTER1,  W10217316,  W10291030,  W10295370,  W10295370A,  W10569758,  W10569760,  W10569761,  W10735398     Back to Top', 'chunk': 0}, {'id': 'PS11701542-1', 'text': 'Water ?? filter needed to be replaced:\n\nSimple, turn and pull to remove. Push and turn to install.\n\n\n\n---\n\nWhen the filter was removed the housing leaked.:\n\nI would suggest loosening the supply lines attached inside the rear. This will give you plenty of slack to disconnect the lines from the housing.                \n                         Other Parts Used: \n                        \n                              

In [35]:
query = "How can I install part number PS11752778?"
# query = "who won the 2010 nba finals?"
# query = "Is this part compatible with my WDT780SAEM1 model?"
# system message to 'prime' the model
primer = f"""You are Q&A bot for an e-commerce site. You are a highly intelligent system that answers
customer questions about various products. You will first be provided with information about the product
the customer is inquiring about including the product description, reviews, general troubleshooting advice, 
repair stories from customers, and popular questions from other customers and their corresponding answers
scraped from the site. Use this information to answer the customer's question about this product.
If the information can not be found in the provided context, you should truthfully say "I don't know".
"""

augmented = augment_query4(query, DB_LOOKUP2)
from openai import OpenAI

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)
res = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": augmented}
    ]
)
from IPython.display import Markdown

display(Markdown(res.choices[0].message.content))

To install part number PS11752778, which is the Refrigerator Door Shelf Bin, you can follow these steps:

1. Locate the area on your refrigerator door where the shelf bin is meant to be installed.
2. Ensure that the shelf bin is positioned correctly, with the front white trim facing outward.
3. Simply slide the shelf bin onto the appropriate supports on the door until it is securely in place.
4. Double-check to make sure the shelf bin is stable and can hold items without any issues.

Since this particular shelf bin is easy to install and requires no tools, you should be able to complete the installation process without much difficulty. If you encounter any specific issues during the installation, feel free to refer to your user manual for further guidance.

In [38]:
def save_dbs(DB_LOOKUP):
    reverse = {}
    joint = {}
    for k, v in DB_LOOKUP.items():
        if v in reverse:
            reverse[v].append(k)
        else:
            reverse[v] = [k]
    for k, v in reverse.items():
        k.save_local('-'.join(v))

In [39]:
save_dbs(DB_LOOKUP2)

### CSV Embeddings

In [ ]:
# search function
def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int = 100
) -> tuple[list[str], list[float]]:
    """Returns a list of strings and relatednesses, sorted from most related to least."""
    query_embedding_response = client.embeddings.create(
        model=EMBEDDING_MODEL,
        input=query,
    )
    query_embedding = query_embedding_response.data[0].embedding
    strings_and_relatednesses = [
        (row["text"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return strings[:top_n], relatednesses[:top_n]
